# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 21 2022 3:18PM,240112,10,0085788511,ISDIN Corporation,Released
1,Jun 21 2022 3:18PM,240112,10,0085788512,ISDIN Corporation,Released
2,Jun 21 2022 3:18PM,240112,10,0085788549,ISDIN Corporation,Released
3,Jun 21 2022 3:18PM,240112,10,0085788591,ISDIN Corporation,Released
4,Jun 21 2022 3:18PM,240112,10,0085788699,ISDIN Corporation,Released
5,Jun 21 2022 3:18PM,240112,10,0085788698,ISDIN Corporation,Released
6,Jun 21 2022 3:18PM,240112,10,0085788710,ISDIN Corporation,Released
7,Jun 21 2022 3:18PM,240112,10,0085788713,ISDIN Corporation,Released
8,Jun 21 2022 3:18PM,240112,10,0085788714,ISDIN Corporation,Released
9,Jun 21 2022 3:18PM,240112,10,0085788721,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,240108,Released,41
39,240109,Released,1
40,240110,Released,4
41,240111,Released,1
42,240112,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240108,NaN,NaN,41.0
240109,NaN,NaN,1.0
240110,NaN,NaN,4.0
240111,NaN,NaN,1.0
240112,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240003,0.0,0.0,1.0
240006,0.0,0.0,1.0
240007,0.0,0.0,1.0
240030,0.0,0.0,1.0
240034,0.0,0.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240003,0,0,1
240006,0,0,1
240007,0,0,1
240030,0,0,1
240034,0,0,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240003,0,0,1
1,240006,0,0,1
2,240007,0,0,1
3,240030,0,0,1
4,240034,0,0,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240003,,,1
1,240006,,,1
2,240007,,,1
3,240030,,,1
4,240034,,,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 21 2022 3:18PM,240112,10,ISDIN Corporation
12,Jun 21 2022 3:10PM,240111,10,ISDIN Corporation
13,Jun 21 2022 3:08PM,240110,10,"Methapharm, Inc."
17,Jun 21 2022 3:06PM,240109,18,"ClearSpec, LLC"
18,Jun 21 2022 3:04PM,240108,10,ISDIN Corporation
59,Jun 21 2022 2:50PM,240106,18,"ClearSpec, LLC"
60,Jun 21 2022 2:46PM,240074,10,"Methapharm, Inc."
62,Jun 21 2022 1:45PM,240103,10,"Snap Medical Industries, LLC"
63,Jun 21 2022 1:43PM,240102,19,"GUSA Granules USA, Inc."
65,Jun 21 2022 1:15PM,240095,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 21 2022 3:18PM,240112,10,ISDIN Corporation,,,12
1,Jun 21 2022 3:10PM,240111,10,ISDIN Corporation,,,1
2,Jun 21 2022 3:08PM,240110,10,"Methapharm, Inc.",,,4
3,Jun 21 2022 3:06PM,240109,18,"ClearSpec, LLC",,,1
4,Jun 21 2022 3:04PM,240108,10,ISDIN Corporation,,,41
5,Jun 21 2022 2:50PM,240106,18,"ClearSpec, LLC",,,1
6,Jun 21 2022 2:46PM,240074,10,"Methapharm, Inc.",,,2
7,Jun 21 2022 1:45PM,240103,10,"Snap Medical Industries, LLC",,,1
8,Jun 21 2022 1:43PM,240102,19,"GUSA Granules USA, Inc.",,,2
9,Jun 21 2022 1:15PM,240095,19,"GCH Granules USA, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 3:18PM,240112,10,ISDIN Corporation,12,,
1,Jun 21 2022 3:10PM,240111,10,ISDIN Corporation,1,,
2,Jun 21 2022 3:08PM,240110,10,"Methapharm, Inc.",4,,
3,Jun 21 2022 3:06PM,240109,18,"ClearSpec, LLC",1,,
4,Jun 21 2022 3:04PM,240108,10,ISDIN Corporation,41,,
5,Jun 21 2022 2:50PM,240106,18,"ClearSpec, LLC",1,,
6,Jun 21 2022 2:46PM,240074,10,"Methapharm, Inc.",2,,
7,Jun 21 2022 1:45PM,240103,10,"Snap Medical Industries, LLC",1,,
8,Jun 21 2022 1:43PM,240102,19,"GUSA Granules USA, Inc.",2,,
9,Jun 21 2022 1:15PM,240095,19,"GCH Granules USA, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 3:18PM,240112,10,ISDIN Corporation,12,,
1,Jun 21 2022 3:10PM,240111,10,ISDIN Corporation,1,,
2,Jun 21 2022 3:08PM,240110,10,"Methapharm, Inc.",4,,
3,Jun 21 2022 3:06PM,240109,18,"ClearSpec, LLC",1,,
4,Jun 21 2022 3:04PM,240108,10,ISDIN Corporation,41,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 3:18PM,240112,10,ISDIN Corporation,12.0,NaN,NaN
1,Jun 21 2022 3:10PM,240111,10,ISDIN Corporation,1.0,NaN,NaN
2,Jun 21 2022 3:08PM,240110,10,"Methapharm, Inc.",4.0,NaN,NaN
3,Jun 21 2022 3:06PM,240109,18,"ClearSpec, LLC",1.0,NaN,NaN
4,Jun 21 2022 3:04PM,240108,10,ISDIN Corporation,41.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 3:18PM,240112,10,ISDIN Corporation,12.0,0.0,0.0
1,Jun 21 2022 3:10PM,240111,10,ISDIN Corporation,1.0,0.0,0.0
2,Jun 21 2022 3:08PM,240110,10,"Methapharm, Inc.",4.0,0.0,0.0
3,Jun 21 2022 3:06PM,240109,18,"ClearSpec, LLC",1.0,0.0,0.0
4,Jun 21 2022 3:04PM,240108,10,ISDIN Corporation,41.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3841182,131.0,18.0,3.0
102,240089,1.0,0.0,0.0
15,1200294,21.0,73.0,0.0
18,480215,2.0,0.0,0.0
19,1200444,5.0,1.0,0.0
20,960259,34.0,33.0,0.0
22,240003,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3841182,131.0,18.0,3.0
1,102,240089,1.0,0.0,0.0
2,15,1200294,21.0,73.0,0.0
3,18,480215,2.0,0.0,0.0
4,19,1200444,5.0,1.0,0.0
5,20,960259,34.0,33.0,0.0
6,22,240003,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,131.0,18.0,3.0
1,102,1.0,0.0,0.0
2,15,21.0,73.0,0.0
3,18,2.0,0.0,0.0
4,19,5.0,1.0,0.0
5,20,34.0,33.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,131.0
1,102,Released,1.0
2,15,Released,21.0
3,18,Released,2.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,18,19,20,22
Status,,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,18.0,0.0,73.0,0.0,1.0,33.0,0.0
Released,131.0,1.0,21.0,2.0,5.0,34.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,18,19,20,22
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,73.0,0.0,1.0,33.0,0.0
2,Released,131.0,1.0,21.0,2.0,5.0,34.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,18,19,20,22
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,73.0,0.0,1.0,33.0,0.0
2,Released,131.0,1.0,21.0,2.0,5.0,34.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()